# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [68]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [69]:
cities_original = pd.read_csv("../output_data/cities.csv")

cities_original.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [70]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [71]:
# Store 'Lat' and 'Lng' into  locations 
cities_original = cities_original.dropna()
locations = cities_original[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values

humidity = cities_original["Humidity"].astype(float)

In [72]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [73]:
#use .loc

#.head()
#caveat: make sure your ideal isn't too ideal, meaning check the length of resulting dataframe so it has rows
#call this dataframe hotel_df

hotel_df=cities_original.loc[(cities_original["Max Temp"] < 85)&(cities_original["Humidity"] < 30)]
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11
42,42,yellowknife,75,CA,1558378762,26,62.45,-114.38,55.40,8.05
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88
156,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [74]:
# set up additional columns to hold information
hotel_df['Hotel Name'] = ""

hotel_df.head()

<ipython-input-74-8544a96b2cb2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11,
42,42,yellowknife,75,CA,1558378762,26,62.45,-114.38,55.40,8.05,
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88,
156,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17,
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,


In [75]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat =row["Lat"]
    lon =row["Lng"]

    # add keyword to params dict
    params['location'] = f'{lat},{lon}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 14: constitucion.
Closest constitucion is 18 De Marzo.
------------
Retrieving Results for Index 42: yellowknife.


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Closest yellowknife is The Explorer Hotel.
------------
Retrieving Results for Index 52: taybad.
Closest taybad is لی پوش jean center.
------------
Retrieving Results for Index 156: narsaq.
Closest narsaq is Narsaq Kayak Hostel.
------------
Retrieving Results for Index 167: yumen.
Closest yumen is 玉门宾馆.
------------
Retrieving Results for Index 187: mackay.
Closest mackay is Nojack Provincial Recreation Area.
------------
Retrieving Results for Index 230: moses lake.
Closest moses lake is Ramada by Wyndham Moses Lake.
------------
Retrieving Results for Index 259: hami.
Closest hami is Post And Telecommunication Hotel.
------------
Retrieving Results for Index 282: slave lake.
Closest slave lake is Super 8 by Wyndham Slave Lake AB.
------------
Retrieving Results for Index 299: jiuquan.
Closest jiuquan is Jiuquan Hotel.
------------
Retrieving Results for Index 324: sidney.
Closest sidney is Holiday Inn Express & Suites Sidney, an IHG Hotel.
------------
Retrieving Results for Index 3

In [76]:
hotel_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11,18 De Marzo
42,42,yellowknife,75,CA,1558378762,26,62.45,-114.38,55.40,8.05,The Explorer Hotel
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88,لی پوش jean center
156,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17,Narsaq Kayak Hostel
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,玉门宾馆
187,187,mackay,20,CA,1558378789,28,53.65,-115.58,59.00,20.80,Nojack Provincial Recreation Area
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87,Ramada by Wyndham Moses Lake
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20,Post And Telecommunication Hotel
282,282,slave lake,1,CA,1558378673,25,55.28,-114.77,62.60,17.22,Super 8 by Wyndham Slave Lake AB
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50,Jiuquan Hotel


In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [78]:
#Convert Hotel df to list
#hotel_rate = hotel_df["Bank Count"].tolist()

In [79]:
# Add marker layer ontop of heat map
# Create bank symbol layer

# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
#                                  dissipating=False, max_intensity=200,
#                                  point_radius = 5)

# fig.add_layer(heat_layer)

# fig


locations = hotel_df['Lat','Lng'] 

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel Name: {column['Hotel Name']}, City:{column['City']} Country:{column['Country']}" 
                      for hotel in hotel_info]

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

# Display figure


SyntaxError: invalid syntax (<ipython-input-79-7164b008a604>, line 23)